# 4.1 Multiplying square matrices

Let us implement all matrix multiplication algorithms described in the book for good measure. As in the book, we pass matrices by reference to our functions, and consider only square integer matrices. As a point of interest, let us make it so that each algorithm returns the number of multiplications and additions of numbers that are made.

As the recursive case requires considering submatrices, we have two options:

1. Create submatrices by allocating new memory and copying entries. This uses more resources, as copying and allocating are considered to be constant-time operations. In this manner, for the matrix $A$,
    - All entries will be copied into submatrices of size $n/2$.
    - All entries will be copied again into submatrices of size $n/4$.
    - All entries will be copied again into submatrices of size $n/8$.
    
    and so on. This adds up to $\Theta(n^2\lg n)$ copies, which is asymptotically negligible.
2. Add more arguments to our function. This is to say that instead of simply giving $A=\begin{bmatrix}a_{ij}\end{bmatrix}_{ij}$ as an argument, we given $A,i_1,i_2,j_1$ and $j_2$, to mean that the argument is in fact the submatrix consisting of the entries of $A$ between rows $i_1$ and $i_2$ and columns $j_1$ and $j_2$. This gets us rid of the need of allocating memory and copying entries, but makes the code less readable.

In [54]:
#include <stdio.h>
#include <time.h>

int matrix_multiply ( int ** A , int ** B , int ** C , size_t n ) {
    int ops = 0; // number of operations
    size_t i,j,k; // indices
    for (i=0;i<n;i++) {
        for (j=0;j<n;j++) {
            for (k=0;k<n;k++) {
                C[i][j] += A[i][k]*B[k][j];
                ops+=2;
            }
        }
    }
    return ops;
}

// In the recursive case, we need to create submatrices. Let us make a function for that

int ** submatrix ( int ** A , size_t i1 , size_t i2 , size_t j1 , size_t j2 ) {
    // Creates a submatrix A[i1:i2,j1:j2] (inclusive)
    size_t i,j; // indices
    size_t m = i2-i1+1;
    size_t n = j2-j1+1;
    
    int ** A_ = malloc(m * sizeof(int *));
    
    for (i=0;i<m;i++) {
        A_[i] = malloc(n*sizeof(int));
        for (j=0;j<n;j++) {
            A_[i][j]=A[i+i1][j+j1];
        }
    }
            
    return A_;
}

// free dinamically allocated matrix for convenience

void free_allocated_matrix(int ** A , size_t m) {
    // Frees a dinamically allocated matrix A with m rows.
    size_t i;
    
    for (i=0;i<m;i++) {
        free(A[i]);
    }
    
    free(A);
}

// print matrix nicely

void print_matrix (int ** A , size_t m , size_t n ) {
    //prints a matrix A in a nice, python-compatible format
    size_t i,j; // indices
    printf("[\n");
    for (i=0;i<m-1;i++) {
        printf("  [");
        for (j=0;j<n-1;j++) {
            printf(" %6d ," , A[i][j]);
        }
        printf(" %6d],\n" , A[i][j]);
    }
    
    // last row
    printf("  [");
    for (j=0;j<n-1;j++) {
        printf(" %6d ," , A[i][j]);
    }
    printf(" %6d]\n" , A[i][j]);
    
    printf("]");
    
}
                   

int matrix_multiply_recursive ( int ** A , int ** B , int ** C , size_t n ) {
    
    //Works only for powers of 2
    
    
    if (n==1) {
        C[0][0] += A[0][0]*B[0][0];
        return 2;
    }
    
    size_t i,j; // Indices
    
    int ops = 0; // number of operations
    
    //Create submatrices
    size_t m = n/2;
    
    int ** A11 = submatrix(A,0,m-1,0,m-1);
    int ** A12 = submatrix(A,0,m-1,m,n-1);
    int ** A21 = submatrix(A,m,n-1,0,m-1);
    int ** A22 = submatrix(A,m,n-1,m,n-1);
    int ** B11 = submatrix(B,0,m-1,0,m-1);
    int ** B12 = submatrix(B,0,m-1,m,n-1);
    int ** B21 = submatrix(B,m,n-1,0,m-1);
    int ** B22 = submatrix(B,m,n-1,m,n-1);
    int ** C11 = submatrix(C,0,m-1,0,m-1);
    int ** C12 = submatrix(C,0,m-1,m,n-1);
    int ** C21 = submatrix(C,m,n-1,0,m-1);
    int ** C22 = submatrix(C,m,n-1,m,n-1);
    
    ops += matrix_multiply_recursive(A11,B11,C11,m);
    ops += matrix_multiply_recursive(A12,B21,C11,m);
    ops += matrix_multiply_recursive(A11,B12,C12,m);
    ops += matrix_multiply_recursive(A12,B22,C12,m);
    ops += matrix_multiply_recursive(A21,B11,C21,m);
    ops += matrix_multiply_recursive(A22,B21,C21,m);
    ops += matrix_multiply_recursive(A21,B12,C22,m);
    ops += matrix_multiply_recursive(A22,B22,C22,m);
    
    // Copy the results back to the original matrices
    
    for (i=0;i<m;i++) {
        for (j=0;j<m;j++) {
            A[i][j] = A11[i][j];
            B[i][j] = B11[i][j];
            C[i][j] = C11[i][j];
        }
        for (;j<n;j++) {
            A[i][j] = A12[i][j-m];
            B[i][j] = B12[i][j-m];
            C[i][j] = C12[i][j-m];
        }
    }
    for (;i<n;i++) {
        for (j=0;j<m;j++) {
            A[i][j] = A21[i-m][j];
            B[i][j] = B21[i-m][j];
            C[i][j] = C21[i-m][j];
        }
        for (;j<n;j++) {
            A[i][j] = A22[i-m][j-m];
            B[i][j] = B22[i-m][j-m];
            C[i][j] = C22[i-m][j-m];
        }
    }    
    
    free_allocated_matrix(A11,m);
    free_allocated_matrix(A12,m);
    free_allocated_matrix(A21,n-m);
    free_allocated_matrix(A22,n-m);
    free_allocated_matrix(B11,m);
    free_allocated_matrix(B12,m);
    free_allocated_matrix(B21,n-m);
    free_allocated_matrix(B22,n-m);
    free_allocated_matrix(C11,m);
    free_allocated_matrix(C12,m);
    free_allocated_matrix(C21,n-m);
    free_allocated_matrix(C22,n-m);
    
    return ops;
}

int main() {
    // Create matrices
    
    srand(time(NULL));
    
    size_t n = 8;
    size_t i,j; // indices
    
    int ** A , ** B , ** C, ** D;
    int number_of_operations;
    
    A = malloc(n*sizeof(int*));
    B = malloc(n*sizeof(int*));
    C = malloc(n*sizeof(int*));
    D = malloc(n*sizeof(int*));
    
    for (i=0;i<n;i++) {
        A[i] = malloc(n*sizeof(int));
        B[i] = malloc(n*sizeof(int));
        C[i] = malloc(n*sizeof(int));
        D[i] = malloc(n*sizeof(int));
        
        for (j=0;j<n;j++) {
            // between -99 and 99
            A[i][j] = rand()%199 - 99;
            B[i][j] = rand()%199 - 99;
            C[i][j] = 0;
            D[i][j] = 0;
        }
    }
    
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");
    
    printf("Random matrices:\n");
    printf("A =\n");
    print_matrix(A,n,n);
    
    printf("\n");
    
    printf("B =\n");
    print_matrix(B,n,n);
    
    // ==============================
    printf("\n");
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");

    printf("Matrix product calculated through standard matrix multiplication:\n");
    
    number_of_operations = matrix_multiply(A,B,C,n);
    
    printf("A*B =\n");
    print_matrix(C,n,n);
    
    printf("\n");
    
    printf("Number of operations: %d" , number_of_operations);
    
    // ==============================
    printf("\n");
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");
    printf("Matrix product calculated through recursive matrix multiplication:\n");
    
    number_of_operations = matrix_multiply_recursive(A,B,D,n);
    
    printf("A*B =\n");
    print_matrix(D,n,n);
    
    printf("\n");
    
    printf("Number of operations: %d" , number_of_operations);
    
    // ==============================
    printf("\n");
    for (i=0;i<30;i++) {
        printf("=");
    }
    printf("\n");
    
    for (i=0;i<n;i++) {
        for (j=0;j<n;j++) {
            if (C[i][j]!=D[i][j]) {
                printf("Entries (i,j)=(%zu,%zu) of the two products are different!\n",i,j);
                i=n+1;
                j=n+1;
            }
        }
    }
    if (i==n && j==n) {
        printf("The two products are the same.");
    }
    printf("\n");
    for (i=0;i<n;i++) {
        free(A[i]);
        free(B[i]);
        free(C[i]);
        free(D[i]);
    }
    
    
    
    free(A);
    free(B);
    free(C);
    free(D);
    
    return 0;
}

Random matrices:
A =
[
  [     17 ,     27 ,    -98 ,    -34 ,    -39 ,    -94 ,     60 ,     15],
  [    -13 ,     64 ,     85 ,    -50 ,    -71 ,     95 ,    -74 ,     48],
  [      3 ,    -24 ,    -30 ,     90 ,     38 ,    -13 ,    -71 ,    -82],
  [     18 ,     75 ,     -9 ,    -18 ,    -89 ,    -11 ,    -22 ,     55],
  [    -12 ,    -61 ,     48 ,    -58 ,     78 ,     85 ,      7 ,     -2],
  [     38 ,    -17 ,    -26 ,    -20 ,    -81 ,     -3 ,     58 ,    -69],
  [    -64 ,     56 ,    -49 ,    -53 ,    -20 ,    -88 ,     88 ,     42],
  [     59 ,    -14 ,     66 ,     -6 ,     10 ,    -43 ,     95 ,    -10]
]
B =
[
  [      2 ,     56 ,     95 ,     10 ,    -79 ,     80 ,    -44 ,     19],
  [     39 ,    -17 ,     32 ,    -98 ,     86 ,     92 ,    -75 ,     42],
  [    -25 ,    -19 ,     41 ,      7 ,     95 ,     -1 ,    -97 ,     15],
  [     59 ,      4 ,    -75 ,     -3 ,    -23 ,    -64 ,    -86 ,     57],
  [    -69 ,     34 ,    -94 ,    -13 ,      5 ,    -93 , 